# 5.5 从 OpenAI 加载预训练权重

之前，出于教育目的，我们使用包含一本短篇故事书的有限数据集训练了一个小型 GPT-2 模型。
这样使我们能够专注于基础知识，而不需要大量的时间和精力。

幸运的是，OpenAI 公开分享了他们的 GPT-2 模型的权重，从而去除了我们自己投资数万到数十万美元在大型语料库上重新训练模型的需要。

在本节的其余部分中，我们将这些权重加载到 GPTModel 类中并使用该模型进行文本生成。
例如，这里的权重是指存储在 PyTorch 的 Linear 和 Embedding 层的 .weight 属性中的权重参数。
我们在训练模型时曾通过model.parameters()访问过它们。

在接下来的章节中，我们将重用这些预训练的权重来对模型进行微调，以便执行文本分类任务，并遵循类似于ChatGPT的指令。

请注意，OpenAI 最初是通过 TensorFlow 保存 GPT-2 的权重的，因此我们需要安装 TensorFlow 来在 Python 中加载这些权重。
此外，下面的代码将使用一个名为 tqdm 的进度条工具来跟踪下载过程，我们也需要安装这个工具。
这些安装步骤确保了在加载模型权重和监控下载进度时的技术兼容性和用户友好性。

您可以通过在终端中执行以下命令来安装这些库：

In [1]:
pip install tensorflow>=2.15.0 tqdm>=4.66

Note: you may need to restart the kernel to use updated packages.


下载代码相对较长，大部分是模板代码，没有太多有趣之处。
因此，我们不在本章中花费宝贵的篇幅讨论从互联网获取文件的Python代码。
相反，我们将直接从本章的在线仓库下载名为gpt_download.py的Python模块。

In [ ]:
import urllib.request
url = (
    "https://raw.githubusercontent.com/rasbt/"
    "LLMs-from-scratch/main/ch05/"
    "01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

接下来，在将此文件下载到Python会话的本地目录后，鼓励读者简要检查这个文件的内容，以确保文件已正确保存并且包含有效的Python代码。

现在，我们可以从 gpt_download.py 文件中导入 download_and_load_gpt2 函数，如下代码所示，该函数会将 GPT-2 架构设置（settings）和权重参数（params）加载到我们的 Python 会话中：

In [ ]:
from gpt_download import download_and_load_gpt2
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

执行后续代码会下载与 124M 参数 GPT-2 模型相关的以下 7 个文件：

checkpoint: 100%|███████████████████████████| 77.0/77.0 [00:00<00:00, 63.9kiB/s] \
encoder.json: 100%|█████████████████████████| 1.04M/1.04M [00:00<00:00, 2.20MiB/s] \
hprams.json: 100%|██████████████████████████| 90.0/90.0 [00:00<00:00, 78.3kiB/s] \
model.ckpt.data-00000-of-00001: 100%|███████| 498M/498M [01:09<00:00, 7.16MiB/s] \
model.ckpt.index: 100%|█████████████████████| 5.21k/5.21k [00:00<00:00, 3.24MiB/s] \
model.ckpt.meta: 100%|██████████████████████| 471k/471k [00:00<00:00, 2.46MiB/s] \
vocab.bpe: 100%|████████████████████████████| 456k/456k [00:00<00:00, 1.70MiB/s]

### 下载指南更新

如果，可能是由于间歇性的互联网连接问题、服务器问题，或者OpenAI分享开源GPT-2模型权重的方式发生了变化,下载代码对您不起作用。
在这种情况下，请访问本章的在线代码仓库 https://github.com/rasbt/LLMs-from-scratch 以获取备用和更新的指南，并请通过Manning论坛联系我们以获取更多帮助。

前面的代码执行完成后，我们来检查一下settings和params的内容：

In [ ]:
print("Settings:", settings)
print("Parameter dictionary keys:", params.keys())

内容如下：

In [ ]:
Settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}
Parameter dictionary keys: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])

settings 和 params 都是Python字典。
settings 字典存储了与我们手动定义的 GPT_CONFIG_124M 设置相似的LLM架构设置。params 字典包含了实际的权重张量。
请注意，我们仅打印了字典键，因为打印权重内容会占用太多屏幕空间，然而，我们可以通过通过 print(params) 打印整个字典或通过相应的字典键选择个别张量来检查这些权重张量，例如，嵌入层权重：

In [ ]:
print(params["wte"])
print("Token embedding weight tensor dimensions:", params["wte"].shape)

token嵌入层的权重如下：

[[-0.11010301 ... -0.1363697 0.01506208 0.04531523] \
[ 0.04034033 ... 0.08605453 0.00253983 0.04318958] \
[-0.12746179 ... 0.08991534 -0.12972379 -0.08785918] \
... \
[-0.04453601 ... 0.10435229 0.09783269 -0.06952604] \
[ 0.1860082 ... -0.09625227 0.07847701 -0.02245961] \
[ 0.05135201 ... 0.00704835 0.15519823 0.12067825]] \
Token embedding weight tensor dimensions: (50257, 768)

我们通过download_and_load_gpt2(model_size="124M", ...)设置下载并加载了最小的GPT-2模型的权重。
然而，请注意，OpenAI还分享了更大模型的权重："355M"、"774M"和"1558M"。这些不同大小的GPT模型的整体架构是相同的，如图5.17所示。

**图5.17 GPT-2大型语言模型有几种不同的模型大小，参数数量从1.24亿到15.58亿不等。
它们的核心架构是相同的，唯一的区别在于嵌入大小以及像注意力头和变换器块这样的单个组件重复的次数。**

![fig5.17](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-5-17.jpg?raw=true)

如图5.17所示，不同大小的GPT-2模型的总体架构保持不变，不同的是各种架构元素重复的次数不同，以及嵌入大小有所不同。
本章剩余的代码也与这些更大的模型兼容。

在将GPT-2模型权重加载到Python之后，我们仍需要将它们从settings和params字典转移到我们的GPTModel实例中。

首先，我们创建一个字典，列出不同 GPT 模型大小之间的差异，如图 5.17 所示：

In [ ]:
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

假设我们有兴趣加载最小的模型“gpt2-small (124M)”。我们可以使用 model_configs 中的相应设置表能够更新我们在本章前面定义和使用的全长 GPT_CONFIG_124M ，操作如下所示：

In [ ]:
model_name = "gpt2-small (124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

细心的读者可能还记得，我们之前设置了256个token的长度，但OpenAI的原始GPT-2模型是定义为1,024个token的长度训练的，因此我们必须相应地更新 NEW_CONFIG：

In [ ]:
NEW_CONFIG.update({"context_length": 1024})

此外，OpenAI在多头注意力模块的线性层中使用偏置向量来实现查询（query）、键（key）和值（value）矩阵的计算。
在大型语言模型（LLM）中，偏置向量已不常使用，因为它们并未提升模型性能，因此是不必要的。
然而，由于我们正在使用预训练的权重，我们需要匹配设置以保持一致性，并启用这些偏置向量：

In [ ]:
NEW_CONFIG.update({"qkv_bias": True})

我们现在可以使用更新后的 NEW_CONFIG 字典来初始化新的 GPTModel 实例：

In [ ]:
gpt = GPTModel(NEW_CONFIG)
gpt.eval()

默认情况下，GPTModel 实例使用随机权重进行初始化以进行预训练。
使用 OpenAI 模型权重的最后一步是用我们加载到 params 字典中的权重覆盖这些随机权重。

为此，我们首先定义一个小的分配实用函数，该函数检查两个张量或数组（左侧和右侧）是否具有相同的维度或形状，并返回右侧张量作为可训练的PyTorch参数：

In [ ]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

接下来，我们定义一个名为 load_weights_into_gpt 函数，该函数将 params 字典中的权重加载到 GPTModel 实例 gpt 中：

### 代码示例5.5 将 OpenAI 权重加载到 GPT 模型代码中

In [ ]:
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe']) #A
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])): #B
        q_w, k_w, v_w = np.split( #C
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight,
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias,
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight,
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias,
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight,
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias,
            params["blocks"][b]["mlp"]["c_proj"]["b"])
        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale,
            params["blocks"][b]["ln_1"]["g"])
        
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift,
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale,
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift,
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"]) #D

在 load_weights_into_gpt 函数中，我们仔细地将 OpenAI 实现的权重与 GPTModel 实现相匹配。
举一个具体的例子，OpenAI 将第一个 Transformer 块的输出投影层的权重张量存储为 params["blocks"][0]["attn"] ["c_proj"]["w"]。
在我们的实现中，这个权重张量对应于gpt.trf_blocks[b].att.out_proj.weight，其中 gpt 是 GPTModel 实例。

由于 OpenAI 使用的命名约定与我们的命名约定略有不同，因此开发 load_weights_into_gpt 函数需要进行大量猜测。
但是，如果我们尝试匹配两个具有不同维度的张量，则assign函数会警告我们。
此外，如果我们在此函数中犯了错误，我们会注意到这一点，因为生成的 GPT 模型将无法生成连贯的文本。

让我们不要只是理论上讨论 load_weights_into_gpt，而是实际操作一下，将 OpenAI 的模型权重加载到我们的 GPTModel 实例 gpt 中：

In [ ]:
load_weights_into_gpt(gpt, params)
gpt.to(device)

如果模型加载正确，我们现在可以使用之前的生成函数来生成新文本：

In [ ]:
torch.manual_seed(123)
token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

输出文本如下：

Output text: \
Every effort moves you toward finding an ideal new way to practice something! \
What makes us want to be on top of that?

我们可以确信模型权重加载正确，因为模型能够生成连贯的文本。
在这个过程中的任何一个微小错误都会导致模型失败。

在接下来的章节中，我们将继续使用这个预训练模型，并对其进行微调，以便进行文本分类和执行指令。

### 练习 5.5

使用 OpenAI 在The Verdict数据集上的预训练权重计算 GPTModel 的训练和验证集损失。

### 练习 5.6

请尝试不同大小的 GPT-2 模型，例如最大的 1558M 参数模型，并将生成的文本与我们在本章中加载的 124M 模型进行比较。